In [2]:
import numpy as np
import pandas as pd
import gc
import os
import json
from collections import Counter, defaultdict
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import re
year_pattern = r'([1-2][0-9]{3})'
pd.set_option('display.max_colwidth', None)

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
df = pd.read_csv('D:\\arxiv dump\\processed.csv')
df.head(10)

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5624\4063950486.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\arxiv dump\\processed.csv')


,Unnamed: 0,id,submitter,title,categories,abstract,update_date,authors_parsed,cat-short,categories_map,new_categories
0,0,704.0001,Pavel Nadolsky,Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies,hep-ph,"A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n",2008-11-26,"[['Balázs', 'C.', ''], ['Berger', 'E. L.', ''], ['Nadolsky', 'P. M.', ''], ['Yuan', 'C. -P.', '']]",hep-ph,High Energy Physics - Phenomenology,['High Energy Physics - Phenomenology']
1,1,704.0002,Louis Theran,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n",2008-12-13,"[['Streinu', 'Ileana', ''], ['Theran', 'Louis', '']]",NaN,NaN,"['Combinatorics', 'Computational Geometry']"
2,2,704.0003,Hongjun Pan,The evolution of the Earth-Moon system based on the dark matter field\n fluid model,physics.gen-ph,"The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n",2008-01-13,"[['Pan', 'Hongjun', '']]",physics.gen-ph,General Physics,['General Physics']
3,3,704.0004,David Callan,A determinant of Stirling cycle numbers counts unlabeled acyclic\n single-source automata,math.CO,We show that a determinant of Stirling cycle numbers counts unlabeled acyclic\nsingle-source automata. The proof involves a bijection from these automata to\ncertain marked lattice paths and a sign-reversing involution to evaluate the\ndeterminant.\n,2007-05-23,"[['Callan', 'David', '']]",math.CO,Combinatorics,['Combinatori

In [ ]:
df["abstract_new"] = df["title"] + df['abstract'].astype(str)
df.head(2)

In [ ]:
import json
data  = []
with open("arxiv-metadata-oai-snapshot.json", 'r') as f:
    for line in f: 
        data.append(json.loads(line))

In [ ]:
data[0: 10]

In [ ]:
df = pd.DataFrame.from_records(data)
df.drop(['authors', 'journal-ref', 'doi', 'comments', 'report-no', 'license', 'versions'], axis = 1, inplace = True) 
df.head(100)

In [62]:
with open("D:\\arxiv dump\\authors-parsed.json", 'r') as f:
     authors = json.load(f)

In [ ]:
new_Author = {}
for key, val in authors.items():
    print(key, val)
    newlist = []
    for value in val:
        newlist.append(" ".join(value))

0704.0001 [['Balázs', 'C.', ''], ['Berger', 'E. L.', ''], ['Nadolsky', 'P. M.', ''], ['Yuan', 'C. -P.', '']]
0704.0002 [['Streinu', 'Ileana', ''], ['Theran', 'Louis', '']]
0704.0003 [['Pan', 'Hongjun', '']]
0704.0004 [['Callan', 'David', '']]
0704.0005 [['Abu-Shammala', 'Wael', ''], ['Torchinsky', 'Alberto', '']]
0704.0006 [['Pong', 'Y. H.', ''], ['Law', 'C. K.', '']]
0704.0007 [['Corichi', 'Alejandro', ''], ['Vukasinac', 'Tatjana', ''], ['Zapata', 'Jose A.', '']]
0704.0008 [['Swift', 'Damian C.', '']]
0704.0009 [['Harvey', 'Paul', ''], ['Merin', 'Bruno', ''], ['Huard', 'Tracy L.', ''], ['Rebull', 'Luisa M.', ''], ['Chapman', 'Nicholas', ''], ['Evans', 'Neal J.', 'II'], ['Myers', 'Philip C.', '']]
0704.0010 [['Ovchinnikov', 'Sergei', '']]
0704.0011 [['Cunningham', 'Clifton', ''], ['Dembele', 'Lassina', '']]
0704.0012 [['Choi', 'Dohoon', '']]
0704.0013 [['Choi', 'Dohoon', ''], ['Choie', 'YoungJu', '']]
0704.0014 [['Fujii', 'Koichi', '']]
0704.0015 [['Stahn', 'Christian', '']]
0704.0016 

In [65]:
import itertools
a = [['a','b'], ['c']]
print(list(itertools.chain.from_iterable(a)))

['a', 'b', 'c']


In [ ]:
# with open("authors-parsed.json", 'r') as f:
#     authors = json.load(f)
with open("internal-citations.json", 'r') as f:
    citations = json.load(f)

In [ ]:
citations

In [ ]:
category_map = {'astro-ph': 'Astrophysics',
'astro-ph.CO': 'Cosmology and Nongalactic Astrophysics',
'astro-ph.EP': 'Earth and Planetary Astrophysics',
'astro-ph.GA': 'Astrophysics of Galaxies',
'astro-ph.HE': 'High Energy Astrophysical Phenomena',
'astro-ph.IM': 'Instrumentation and Methods for Astrophysics',
'astro-ph.SR': 'Solar and Stellar Astrophysics',
'cond-mat.dis-nn': 'Disordered Systems and Neural Networks',
'cond-mat.mes-hall': 'Mesoscale and Nanoscale Physics',
'cond-mat.mtrl-sci': 'Materials Science',
'cond-mat.other': 'Other Condensed Matter',
'cond-mat.quant-gas': 'Quantum Gases',
'cond-mat.soft': 'Soft Condensed Matter',
'cond-mat.stat-mech': 'Statistical Mechanics',
'cond-mat.str-el': 'Strongly Correlated Electrons',
'cond-mat.supr-con': 'Superconductivity',
'cs.AI': 'Artificial Intelligence',
'cs.AR': 'Hardware Architecture',
'cs.CC': 'Computational Complexity',
'cs.CE': 'Computational Engineering, Finance, and Science',
'cs.CG': 'Computational Geometry',
'cs.CL': 'Computation and Language',
'cs.CR': 'Cryptography and Security',
'cs.CV': 'Computer Vision and Pattern Recognition',
'cs.CY': 'Computers and Society',
'cs.DB': 'Databases',
'cs.DC': 'Distributed, Parallel, and Cluster Computing',
'cs.DL': 'Digital Libraries',
'cs.DM': 'Discrete Mathematics',
'cs.DS': 'Data Structures and Algorithms',
'cs.ET': 'Emerging Technologies',
'cs.FL': 'Formal Languages and Automata Theory',
'cs.GL': 'General Literature',
'cs.GR': 'Graphics',
'cs.GT': 'Computer Science and Game Theory',
'cs.HC': 'Human-Computer Interaction',
'cs.IR': 'Information Retrieval',
'cs.IT': 'Information Theory',
'cs.LG': 'Machine Learning',
'cs.LO': 'Logic in Computer Science',
'cs.MA': 'Multiagent Systems',
'cs.MM': 'Multimedia',
'cs.MS': 'Mathematical Software',
'cs.NA': 'Numerical Analysis',
'cs.NE': 'Neural and Evolutionary Computing',
'cs.NI': 'Networking and Internet Architecture',
'cs.OH': 'Other Computer Science',
'cs.OS': 'Operating Systems',
'cs.PF': 'Performance',
'cs.PL': 'Programming Languages',
'cs.RO': 'Robotics',
'cs.SC': 'Symbolic Computation',
'cs.SD': 'Sound',
'cs.SE': 'Software Engineering',
'cs.SI': 'Social and Information Networks',
'cs.SY': 'Systems and Control',
'econ.EM': 'Econometrics',
'eess.AS': 'Audio and Speech Processing',
'eess.IV': 'Image and Video Processing',
'eess.SP': 'Signal Processing',
'gr-qc': 'General Relativity and Quantum Cosmology',
'hep-ex': 'High Energy Physics - Experiment',
'hep-lat': 'High Energy Physics - Lattice',
'hep-ph': 'High Energy Physics - Phenomenology',
'hep-th': 'High Energy Physics - Theory',
'math.AC': 'Commutative Algebra',
'math.AG': 'Algebraic Geometry',
'math.AP': 'Analysis of PDEs',
'math.AT': 'Algebraic Topology',
'math.CA': 'Classical Analysis and ODEs',
'math.CO': 'Combinatorics',
'math.CT': 'Category Theory',
'math.CV': 'Complex Variables',
'math.DG': 'Differential Geometry',
'math.DS': 'Dynamical Systems',
'math.FA': 'Functional Analysis',
'math.GM': 'General Mathematics',
'math.GN': 'General Topology',
'math.GR': 'Group Theory',
'math.GT': 'Geometric Topology',
'math.HO': 'History and Overview',
'math.IT': 'Information Theory',
'math.KT': 'K-Theory and Homology',
'math.LO': 'Logic',
'math.MG': 'Metric Geometry',
'math.MP': 'Mathematical Physics',
'math.NA': 'Numerical Analysis',
'math.NT': 'Number Theory',
'math.OA': 'Operator Algebras',
'math.OC': 'Optimization and Control',
'math.PR': 'Probability',
'math.QA': 'Quantum Algebra',
'math.RA': 'Rings and Algebras',
'math.RT': 'Representation Theory',
'math.SG': 'Symplectic Geometry',
'math.SP': 'Spectral Theory',
'math.ST': 'Statistics Theory',
'math-ph': 'Mathematical Physics',
'nlin.AO': 'Adaptation and Self-Organizing Systems',
'nlin.CD': 'Chaotic Dynamics',
'nlin.CG': 'Cellular Automata and Lattice Gases',
'nlin.PS': 'Pattern Formation and Solitons',
'nlin.SI': 'Exactly Solvable and Integrable Systems',
'nucl-ex': 'Nuclear Experiment',
'nucl-th': 'Nuclear Theory',
'physics.acc-ph': 'Accelerator Physics',
'physics.ao-ph': 'Atmospheric and Oceanic Physics',
'physics.app-ph': 'Applied Physics',
'physics.atm-clus': 'Atomic and Molecular Clusters',
'physics.atom-ph': 'Atomic Physics',
'physics.bio-ph': 'Biological Physics',
'physics.chem-ph': 'Chemical Physics',
'physics.class-ph': 'Classical Physics',
'physics.comp-ph': 'Computational Physics',
'physics.data-an': 'Data Analysis, Statistics and Probability',
'physics.ed-ph': 'Physics Education',
'physics.flu-dyn': 'Fluid Dynamics',
'physics.gen-ph': 'General Physics',
'physics.geo-ph': 'Geophysics',
'physics.hist-ph': 'History and Philosophy of Physics',
'physics.ins-det': 'Instrumentation and Detectors',
'physics.med-ph': 'Medical Physics',
'physics.optics': 'Optics',
'physics.plasm-ph': 'Plasma Physics',
'physics.pop-ph': 'Popular Physics',
'physics.soc-ph': 'Physics and Society',
'physics.space-ph': 'Space Physics',
'q-bio.BM': 'Biomolecules',
'q-bio.CB': 'Cell Behavior',
'q-bio.GN': 'Genomics',
'q-bio.MN': 'Molecular Networks',
'q-bio.NC': 'Neurons and Cognition',
'q-bio.OT': 'Other Quantitative Biology',
'q-bio.PE': 'Populations and Evolution',
'q-bio.QM': 'Quantitative Methods',
'q-bio.SC': 'Subcellular Processes',
'q-bio.TO': 'Tissues and Organs',
'q-fin.CP': 'Computational Finance',
'q-fin.EC': 'Economics',
'q-fin.GN': 'General Finance',
'q-fin.MF': 'Mathematical Finance',
'q-fin.PM': 'Portfolio Management',
'q-fin.PR': 'Pricing of Securities',
'q-fin.RM': 'Risk Management',
'q-fin.ST': 'Statistical Finance',
'q-fin.TR': 'Trading and Market Microstructure',
'quant-ph': 'Quantum Physics',
'stat.AP': 'Applications',
'stat.CO': 'Computation',
'stat.ME': 'Methodology',
'stat.ML': 'Machine Learning',
'stat.OT': 'Other Statistics',
'stat.TH': 'Statistics Theory'}

In [ ]:
categories_DF = pd.DataFrame(category_map.items(), columns=['cat-short', 'categories_map'])
categories_DF.head(10)

In [ ]:
df = pd.merge(df,categories_DF, left_on='categories', right_on='cat-short' , how='left')
df.head(20)

In [ ]:
df["categories_lst"]= df["categories"].str.split(" ", n = 1) 
df.head(20)

In [10]:
sample = df[0:200]
sample.head(10)

,Unnamed: 0,abstract
0,0,"Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n"
1,1,"Sparsity-certifying Graph Decompositions We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n"
2,2,"The evolution of the Earth-Moon system based on the dark matter field\n fluid model The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n"
3,3,A determinant of Stirling cycle numbers counts unlabeled acyclic\n single-source automata We show that a determinant of Stirling cycle numbers counts unlabeled acyclic\nsingle-source automata. The proof involves a bijection from these automata to\ncertain marked lattice paths and a sign-reversing involution to evaluate the\ndeterminant.\n
4,4,"From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\alpha}$ In this paper we show how to compute the $\Lambda_{\alpha}$ norm, $\alpha\ge\n0$, using the dyadic grid. This result is a consequence of the description of\nthe Hardy spaces $H^p(R^N)$ in terms of dyadic and special atoms.\n"
5,5,"Bosonic characters of atomic Cooper pairs across resonance We study the two-particle wave function of paired atoms in a Fermi gas with\ntunable interaction strengths controlled by Feshbach resonance. The Cooper pair\nwave function is examined for its bosonic characters, which is quantified by\nthe correction of Bose enhancement factor associated with the creation and\nannihilation composite particle operators. An ex

In [ ]:
values = []
for index, row in df.iterrows():
     lst = row["categories"].split()
     eachLst =[]   
     for each in lst:
        eachLst.append(category_map.get(each, "empty"))
     values.append(eachLst)   

In [ ]:
df2 = pd.DataFrame().assign(abstract=df['abstract_new'])
df2.head(10)

In [5]:
df2.to_csv("concatTitleAndAbstract.csv")

In [6]:
df = pd.read_csv('D:\\arxiv dump\\concatTitleAndAbstract.csv')
df.head(2)

,Unnamed: 0,abstract
0,0,"Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n"
1,1,"Sparsity-certifying Graph Decompositions We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n"
2,2,"The evolution of the Earth-Moon system based on the dark matter field\n fluid model The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n"
3,3,A determinant of Stirling cycle numbers counts unlabeled acyclic\n single-source automata We show that a determinant of Stirling cycle numbers counts unlabeled acyclic\nsingle-source automata. The proof involves a bijection from these automata to\ncertain marked lattice paths and a sign-reversing involution to evaluate the\ndeterminant.\n
4,4,"From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\alpha}$ In this paper we show how to compute the $\Lambda_{\alpha}$ norm, $\alpha\ge\n0$, using the dyadic grid. This result is a consequence of the description of\nthe Hardy spaces $H^p(R^N)$ in terms of dyadic and special atoms.\n"
5,5,"Bosonic characters of atomic Cooper pairs across resonance We study the two-particle wave function of paired atoms in a Fermi gas with\ntunable interaction strengths controlled by Feshbach resonance. The Cooper pair\nwave function is examined for its bosonic characters, which is quantified by\nthe correction of Bose enhancement factor associated with the creation and\nannihilation composite particle operators. An ex

In [6]:
df.count()

Unnamed: 0        2120481
id                2120481
submitter         2105291
title             2120481
categories        2120481
abstract          2120481
update_date       2120481
authors_parsed    2120481
cat-short         1159353
categories_map    1159353
new_categories    2120481
dtype: int64

In [ ]:
from transformers import AutoTokenizer, TFAutoModel

# model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
model_ckpt = "sentence-transformers/all-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True)

In [8]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [9]:
import torch
def get_embeddings(text_list):
    encoded_input = tokenizer(text_list, padding=True, truncation=True, return_tensors="tf")
    encoded_input = {k: v for k, v in encoded_input.items()}
    with torch.no_grad():
        model_output = model(**encoded_input)
    # return cls_pooling(model_output)
    # return  mean_pooling(model_output, encoded_input["attention_mask"])
    return cls_pooling(model_output)

In [11]:
embeddings_dataset= comments_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["abstract"]).numpy()[0]}
)
embeddings_dataset

  0%|          | 0/1000 [00:00<?, ?ex/s]

Dataset({
    features: ['Unnamed: 0', 'abstract', 'embeddings'],
    num_rows: 1000
})

In [10]:
from datasets import Dataset
comments_dataset = Dataset.from_pandas(df[0:1000])
comments_dataset

Dataset({
    features: ['Unnamed: 0', 'abstract'],
    num_rows: 1000
})

Sequence(feature='float64', length=-1, id=None)

## Load Model

In [ ]:
from datasets import load_dataset
from datasets import ClassLabel, Value, Sequence, Features
emotion_features = Features({'embeddings': Sequence('float64')})
dataset = load_dataset("csv", data_files="Embeddings/Test-0.csv", features = emotion_features)
dataset


In [12]:
embeddings_dataset.add_faiss_index(column="embeddings")

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['Unnamed: 0', 'abstract', 'embeddings'],
    num_rows: 1000
})

In [13]:
question = "Polymer Quantum Mechanics and its Continuum Limit A rather non-standard quantum representation of the canonicale"
question_embedding = get_embeddings([question]).numpy()

scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=3
)

In [14]:
scores

array([10.957794, 22.521162, 23.011349], dtype=float32)

In [15]:
samples

{'Unnamed: 0': [6, 287, 706],
 'abstract': ['Polymer Quantum Mechanics and its Continuum Limit  A rather non-standard quantum representation of the canonical commutation\nrelations of quantum mechanics systems, known as the polymer representation has\ngained some attention in recent years, due to its possible relation with Planck\nscale physics. In particular, this approach has been followed in a symmetric\nsector of loop quantum gravity known as loop quantum cosmology. Here we explore\ndifferent aspects of the relation between the ordinary Schroedinger theory and\nthe polymer description. The paper has two parts. In the first one, we derive\nthe polymer quantum mechanics starting from the ordinary Schroedinger theory\nand show that the polymer description arises as an appropriate limit. In the\nsecond part we consider the continuum limit of this theory, namely, the reverse\nprocess in which one starts from the discrete theory and tries to recover back\nthe ordinary Schroedinger quantu

In [ ]:
embeddings_dataset['features']

In [96]:
embeddings_dataset[

In [119]:
df11_dataset = Dataset.from_pandas(df11[0:200])
df11_dataset

Dataset({
    features: ['Unnamed: 0', 'abstract', 'embeddings'],
    num_rows: 200
})

In [21]:
from datasets import load_dataset
from datasets import ClassLabel, Value, Sequence, Features
emotion_features = Features({'embeddings': Sequence('float64')})
dataset = load_dataset("csv", data_files="D:\\arxiv dump\\Embeddings/Test-0.csv")
dataset

Using custom data configuration default-e03fb249ac2a2e54
Reusing dataset csv (C:\Users\sumeet\.cache\huggingface\datasets\csv\default-e03fb249ac2a2e54\0.0.0\e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23)


DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'abstract', 'embeddings'],
        num_rows: 100000
    })
})

Dataset({
    features: ['Unnamed: 0', 'abstract', 'embeddings'],
    num_rows: 100000
})

In [ ]:
dataset = pd.read_csv("D:\\arxiv dump\\Embeddings/Test-0.csv")


In [9]:
df = pd.read_csv('D:\\arxiv dump\\processed.csv')
df = df[['Unnamed: 0', 'id', 'title', 'abstract', 'update_date', 'authors_parsed', 'new_categories']]
df.rename(columns={'Unnamed: 0': 'order',
                   'authors_parsed': 'authors',
                   'new_categories': 'Categories'},
          inplace=True, errors='raise')
df.head(3)

C:\Users\sumeet\AppData\Local\Temp\ipykernel_5624\1844388970.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:\\arxiv dump\\processed.csv')


,order,id,title,abstract,update_date,authors,Categories
0,0,704.0001,Calculation of prompt diphoton production cross sections at Tevatron and\n LHC energies,"A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from the Fermilab Tevatron, and predictions are made for\nmore detailed tests with CDF and DO data. Predictions are shown for\ndistributions of diphoton pairs produced at the energy of the Large Hadron\nCollider (LHC). Distributions of the diphoton pairs from the decay of a Higgs\nboson are contrasted with those produced from QCD processes at the LHC, showing\nthat enhanced sensitivity to the signal can be obtained with judicious\nselection of events.\n",2008-11-26,"[['Balázs', 'C.', ''], ['Berger', 'E. L.', ''], ['Nadolsky', 'P. M.', ''], ['Yuan', 'C. -P.', '']]",['High Energy Physics - Phenomenology']
1,1,704.0002,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-pebble game with colors, and use\nit obtain a characterization of the family of $(k,\ell)$-sparse graphs and\nalgorithmic solutions to a family of problems concerning tree decompositions of\ngraphs. Special instances of sparse graphs appear in rigidity theory and have\nreceived increased attention in recent years. In particular, our colored\npebbles generalize and strengthen the previous results of Lee and Streinu and\ngive a new proof of the Tutte-Nash-Williams characterization of arboricity. We\nalso present a new decomposition that certifies sparsity based on the\n$(k,\ell)$-pebble game with colors. Our work also exposes connections between\npebble game algorithms and previous sparse graph algorithms by Gabow, Gabow and\nWestermann and Hendrickson.\n",2008-12-13,"[['Streinu', 'Ileana', ''], ['Theran', 'Louis', '']]","['Combinatorics', 'Computational Geometry']"
2,2,704.0003,The evolution of the Earth-Moon system based on the dark matter field\n fluid model,"The evolution of Earth-Moon system is described by the dark matter field\nfluid model proposed in the Meeting of Division of Particle and Field 2004,\nAmerican Physical Society. The current behavior of the Earth-Moon system agrees\nwith this model very well and the general pattern of the evolution of the\nMoon-Earth system described by this model agrees with geological and fossil\nevidence. The closest distance of the Moon to Earth was about 259000 km at 4.5\nbillion years ago, which is far beyond the Roche's limit. The result suggests\nthat the tidal friction may not be the primary cause for the evolution of the\nEarth-Moon system. The average dark matter field fluid constant derived from\nEarth-Moon system data is 4.39 x 10^(-22) s^(-1)m^(-1). This model predicts\nthat the Mars's rotation is also slowing with the angular acceleration rate\nabout -4.38 x 10^(-22) rad s^(-2).\n",2008-01-13,"[['Pan', 'Hongjun', '']]",['General Physics']


In [ ]:
row1 = df.sample(n = 20)
row1 

In [ ]:
 for index, row in row1.iterrows():
     print(len(row["Categories"]))

In [10]:
for i in range (0, 21):
    temp = "D:\\arxiv dump\\Embeddings\\Test-{0}.csv".format(i)
    embeddingDF = pd.read_csv(temp)
    embeddingDF = embeddingDF[['embeddings']]
    min = int(str(i) + "00000")
    max = int(str(i+1)  + "00000")
    newdf = df[min:max]
    newdf = newdf.reset_index(drop=True)
    
    merged_DF = pd.concat([newdf, embeddingDF], axis = 1)
    #merged_DF.head(5)
    #merged_DF.tail(5)
    merged_DF.to_csv('D:\\arxiv dump\\Embeddings\\merged_out_{0}.csv'.format(i), index=False)
    print(" Processing from ", min, " to ", max)
    print(temp)
    

 Processing from  0  to  100000
D:\arxiv dump\Embeddings\Test-0.csv
 Processing from  100000  to  200000
D:\arxiv dump\Embeddings\Test-1.csv
 Processing from  200000  to  300000
D:\arxiv dump\Embeddings\Test-2.csv
 Processing from  300000  to  400000
D:\arxiv dump\Embeddings\Test-3.csv
 Processing from  400000  to  500000
D:\arxiv dump\Embeddings\Test-4.csv
 Processing from  500000  to  600000
D:\arxiv dump\Embeddings\Test-5.csv
 Processing from  600000  to  700000
D:\arxiv dump\Embeddings\Test-6.csv
 Processing from  700000  to  800000
D:\arxiv dump\Embeddings\Test-7.csv
 Processing from  800000  to  900000
D:\arxiv dump\Embeddings\Test-8.csv
 Processing from  900000  to  1000000
D:\arxiv dump\Embeddings\Test-9.csv
 Processing from  1000000  to  1100000
D:\arxiv dump\Embeddings\Test-10.csv
 Processing from  1100000  to  1200000
D:\arxiv dump\Embeddings\Test-11.csv
 Processing from  1200000  to  1300000
D:\arxiv dump\Embeddings\Test-12.csv
 Processing from  1300000  to  1400000
D:\arxi

In [11]:
merged = pd.read_csv('D:\\arxiv dump\\Embeddings\\merged_out_15.csv')
merged.head(2)

order          id  \
0  1500000  2107.05988   
1  1500001  2107.05989   

                                                                                        title  \
0                       Modelling the Dynamics of Potentially Dangerous Large-Earth Impactors   
1  Emotion Recognition for Healthcare Surveillance Systems Using Neural\n  Networks: A Survey   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  abstract  \
0    Collisional threats posed by Near-Earth Objects (NEOs) are increasingly being\nconfirmed by National Aeronautics and Space Administration (NASA) and European\nSpace Agency (ESA) sky surveys. Efforts to develop tools to perform modelling,\nanalysis and prediction of possible future impact events are ongoing. The aim\nof this research is to design a model for Large Earth impact events, describing\natmospheric entry, ground impact and post-impact dynamics for impactors large\nenough to cause harm to the Earth. We present results of Large Earth Impact\n(LEI) events involving a number of objects with densities ranging between $1000\n- 8000 kg m^{-3}$. We use Python code to solve differential equations and\nperform calculations on analytical models built into the collision simulation\nalgorithms while feeding relevant physical input in terms of object velocity\nand diameter. We find that denser impactors have slower rates of decrease in\nmomentum while hitting the atmosphere. The denser impactors which exhibit\ngreater resistance to atmospheric disruption result in more energetic ground\nimpacts events in terms of crater formation, thermal radiation generation,\nseismic effects, ejecta displacement and air-blast effects. We conclude that\nimpacting objects with high masses maintain greater kinetic energies compared\nto objects with less mass thus posing greater collisional threats to the Earth.\n   
1                                                                                                                                                                                                                                                                                                       Recognizing the patient's emotions using deep learning techniques has\nattracted significant attention recently due to technological advancements.\nAutomatically identifying the emotions can help build smart healthcare centers\nthat can detect depression and stress among the patients in order to start the\nmedication early. Using advanced technology to identify emotions is one of the\nmost exciting topics as it defines the relationships between humans and\nmachines. Machines learned how to

In [12]:
merged.tail(2)

order          id  \
99998  1599998  2202.01174   
99999  1599999  2202.01175   

                                                                                              title  \
99998                                                 Evitable iterates of the consistency operator   
99999  Surface acoustic waves inside polystyrene microparticles through\n  photoacoustic microscopy   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               abstract  \
99998    Let's fix a reasonable subsystem $T$ of arithmetic; why are natural\nextensions of $T$ pre-well-ordered by consistency strength? In previous work,\nan approach to this question was proposed. The goal of this work was to\nclassify the recursive functions that are monotone with respect to the\nLindenabum algebra of $T$. According to an optimistic conjecture, roughly,\nevery such function must be equivalent to an iterate $\mathsf{Con}_T^\alpha$ of\nthe consistency operator in the limit.\n  In previous work the author established the first case of this optimistic\nconjecture; roughly, every recursive monotone function is either as weak as the\nidentity operator in the limit or as strong as $\mathsf{Con}_T$ in the limit.\nYet in this note we prove that this optimistic conjecture fails already at the\nnext step; there are recursive monotone functions that are neither as weak as\n$\mathsf{Con}_T$ in the limit nor as strong as $\mathsf{Con}_T^2$ in the limit.\nIn fact, for every $\alpha$, we produce a function that is cofinally as strong\nas $\mathsf{Con}^\alpha_T$ yet cofinally as weak as $\mathsf{Con}_T$.\n   
99999                                                                                                                                                                                                                                                                                                                               We demonstrate surface acoustic waves inside polystyrene microspheres of\ndifferent sizes experimentally through photoacoustic microscopy and validate\nthe experimental result with simulation. A novel method for sample preparation\nof a lifted sample is also presented where the microparticles are suspended in\nagarose above the surface of the petridish. Another objective of this study was\nto investigate the results with different laser focus hitting on the\nmicroparticles and their impact on photoacoustic images and photoacoustic\nsignals. An absorbing microsphere is excited with a pulsed laser of wavelength\n532 nm and the photoacoustic signal is detected using a 40 MHz transducer. On\nanalyzing the photoacoustic signals from microspheres, we find the signature of\nsurface acoustic waves.\n   

      update_date  \
99998  2022-02-03   
99999  2022-03-22   

                                                                                                                                               authors

In [55]:
merged.authors[0]

'[[\'Muthini\', \'Jude\', \'\'], ["Okeng\'o", \'Geoffrey O.\', \'\']]'

In [56]:
len(merged.authors[0])

57